Information need to retrieve:
1. Author’s association with repository. (author_association) - done previously
2. Flag of whether the comment author also posted the original issue. (is_poster)

3. Length of the sentence in terms of character count. (quote_len)
4. Count of words in sentence divided by that of the longest sentence in thread. (percent_len_thread)
5. Count of words in sentence divided by that of the longest sentence in comment. (percent_len_comment)
6. Position of sentence in comment divided by the number of sentences in comment. (percent_pos_thread)
7. Position of sentence in conversation divided by the number of sentences in thread. (percent_pos_comment)
8. Flag of whether if this is in the first comment. (is_first_comment)
9. Flag of whether this is the last comment or not. (is_last_comment)

10. Time from beginning of conversation to comment divided by the total time of thread. (percent_start_time_thread)
11. Time from comment to end of conversation divided by the total time of thread. (percent_end_time_thread)
12. Time from previous comment to current comment (normalized). (percent_previous_time_thread)
13. Time from current comment to next comment (normalized). (percent_next_time_thread)

14. Flag to indicate whether the comment contains a code snippet. (contain_code)

In [1]:
import numpy as np
import pandas as pd
import re
import requests
import json
from datetime import datetime

In [2]:
datetime_format_github = '%Y-%m-%dT%H:%M:%SZ'

In [3]:
df_quote = pd.read_csv('AuthorAssociation_data.csv')

In [4]:
df_time_author = pd.read_excel('TimeAuthor.xlsx', sheet_name='Sheet1', usecols =['Text Content', 'Document', 'Codes', 'Start position', 'End position'])

In [5]:
def clean_text(line):
    ## Replace all code blocks with the token CODE
    pattern = re.compile(r'```[^```]*?(```|$)', re.MULTILINE|re.DOTALL)
    line = re.sub(pattern,'CODE_BLOCK',line)
    pattern = re.compile(r'`[^`]*?`', re.MULTILINE|re.DOTALL)
    line = re.sub(pattern,'CODE_SEGMENT',line)
    pattern = re.compile(r'(^|\n)>.*')
    line = re.sub(pattern,'\nQUOTE',line)
    pattern = re.compile(r'On [\d]+ \w+ [\d]+ at [\d]+:[\d]+, \w+ \w+ <.*?> wrote:')
    line = re.sub(pattern,'QUOTE',line)
    pattern = re.compile(r'On \w+, \w+ \d+, \d+, \d+:\d+ (A|P)M \w+ \w+ .*? wrote:')
    line = re.sub(pattern,'QUOTE',line)
    pattern = re.compile(r'([\n\t\s]*?QUOTE)+',re.MULTILINE|re.DOTALL)
    line = re.sub(pattern,'\nQUOTE',line)
    ## Replace URLs which are surrounded by brackets
    pattern = re.compile(r'[\[\(]http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+[\]\)]')
    line = re.sub(pattern,' URL ',line)
    pattern = re.compile(r'\B@\w+')
    line = re.sub(pattern,'SCREEN_NAME ',line)
    pattern = re.compile(r'\b([v])?([1-2]+\.)+(\d+\.)?(\*|\d+)\b')
    line = re.sub(pattern,'VERSION_NUM ',line)
    pattern = re.compile(r'\+(\d+)')
    line = re.sub(pattern,'PLUS_ONE',line)
    pattern = re.compile(r'\-(\d+)')
    line = re.sub(pattern,'MINUS_ONE',line)
    pattern = re.compile(r'\#(\d+)')
    line = re.sub(pattern,'ISSUE_REFERENCE',line)
    return line

In [6]:
df_quote['Text Content'] = df_quote['Text Content'].map(lambda row : clean_text(row))

In [7]:
# remove the data point with only non-ascii chars (e.g. only emoji, gibberish chinese)
def GetContainAlphabet(row):
    content = row['Text Content']
    pattern = re.compile(r'[a-zA-Z]+')
    alphabet_content = re.findall(pattern,content)
    if len(alphabet_content) > 0:
        return True
    else:
        return False

In [9]:
df_quote['contain_alphabet'] = df_quote.apply(GetContainAlphabet , axis=1)

In [10]:
df_quote[df_quote['contain_alphabet'] == False]

,Text Content,Document,Codes,Start position,End position,time_pre,author_pre,time_cur,author_cur,time_next,author_next,author_association,contain_alphabet
95,???,issue9388_allCommentsText.doc,"9388_irrelevant, small talk",16103,16106,2017-08-10T13:07:06Z,ShayBox,2017-08-16T18:24:19Z,R3V07T,2017-08-17T07:06:27Z,dinhtuan1991vt,NONE,False
176,ðª,issue9388_allCommentsText.doc,"9388_irrelevant, like it",27797,27801,2018-01-29T14:25:04Z,Montago,2018-01-29T14:26:16Z,luispabon,2018-01-29T14:35:43Z,leonardonelson91,NONE,False
226,ð 1,issue9388_allCommentsText.doc,"9388_irrelevant, like it",20244,20250,2017-10-26T01:18:19Z,technoiswatchingyou,2017-10-27T08:26:01Z,mybios,2017-11-18T07:29:16Z,giangcoi48k,NONE,False
276,è¿ä¸¤ä¸ªè®¾ç½®å¹¶ä¸è½è§£å³é®é¢ï¼æµªè´¹äºèå­å¥½...,issue9388_allCommentsText.doc,9388_irrelevant,18626,18692,2017-09-29T09:36:09Z,svdHero,2017-10-11T08:23:03Z,2289034325,2017-10-14T09:42:31Z,ShayBox,NONE,False
282,ð,issue9388_allCommentsText.doc,9388_irrelevant,17412,17416,2017-08-17T19:10:59Z,ShayBox,2017-08-22T15:22:46Z,chriscarpenter12,2017-08-28T03:34:40Z,superTechnologist,NONE,False
290,__//_/|/â-//â-//â-//â-//â-//Â´Â¯/'--'/Â´Â¯`Â·_...,issue9388_allCommentsText.doc,9388_irrelevant,18693,18876,2017-09-29T09:36:09Z,svdHero,2017-10-11T08:23:03Z,2289034325,2017-10-14T09:42:31Z,ShayBox,NONE,False
637,ð,issue4865_allCommentsText.doc,4865_irrelevant,45950,45954,2017-03-02T09:55:12Z,sixenvi,2017-03-02T13:07:24Z,igoreliasm,2017-03-02T14:12:53Z,ghiscoding,NONE,False
714,ð,issue14909_allCommentsText.doc,"14909_irrelevant, like it",9476,9480,2017-04-10T08:22:52Z,panzer-planet,2017-04-26T16:44:42Z,nicholaswagner,2017-05-29T15:19:31Z,eduncan911,NONE,False
905,ð,issue14909_allCommentsText.doc,"14909_irrelevant, like it",37384,37388,2018-01-18T19:48:27Z,andradei,2018-01-24T00:19:01Z,colinweight,2018-01-24T03:02:25Z,38b3eff8,NONE,False
1187,ð,issue14909_allCommentsText.doc,14909_irrelevant,6467,6468,2017-02-22T22:33:29Z,kieferrm,2017-02-23T17:59:26Z,seanhealy,2017-03-03T22:14:50Z,panzer-planet,NONE,False


In [11]:
display(df_quote.shape)
df_quote = df_quote.loc[df_quote['contain_alphabet'] == True]

display(df_quote.shape)

(5142, 13)

(5123, 13)

In [12]:
df_first_comment = df_time_author.sort_values(['Start position'],ascending=True).groupby('Document').head(2)
df_first_comment = df_first_comment.sort_values(['Start position'],ascending=True).groupby('Document').tail(1)

In [13]:
display(df_first_comment)

,Text Content,Document,Codes,Start position,End position
772,2015-11-20T00:20:01Z inakianduaga,issue224_allCommentsText.doc,time_author,413,446
387,2015-11-21T07:13:47Z i5ting,issue396_allCommentsText.doc,time_author,543,570
89,2016-04-04T12:08:50Z alexandrudima,issue4865_allCommentsText.doc,time_author,577,611
1,2016-07-16T16:20:03Z brunnopleffken,issue9388_allCommentsText.doc,time_author,722,757
195,2016-11-03T14:49:49Z bpasero,issue14909_allCommentsText.doc,time_author,848,876


In [14]:
# original issue poster
def GetPoster(row):
    doc_name = row['Document']
    df_temp_doc = df_time_author.loc[(df_time_author['Document'] == doc_name)]
    df_temp_poster = df_temp_doc.loc[df_temp_doc['Start position'].idxmin()]
    df_temp_poster = df_temp_poster['Text Content'].split()
    return df_temp_poster[0], df_temp_poster[1]

In [15]:
def GetIsFirstComment(row):
    doc_name = row['Document']
    temp_df = df_first_comment.loc[df_first_comment['Document'] == doc_name]
    temp_df = temp_df.iloc[0]
    time_author_list = temp_df['Text Content'].split()
    first_comment_time = time_author_list[0]
    first_comment_author = time_author_list[1]
    return (row['time_cur'] == first_comment_time) & (row['author_cur'] == first_comment_author)

In [16]:
df_quote[['poster_time', 'poster_name']]  = df_quote.apply(GetPoster, axis=1, result_type="expand")

In [17]:
df_quote['is_poster'] = df_quote.apply(lambda row: row['author_cur'] == row['poster_name'] , axis=1)

In [18]:
df_quote['quote_len'] = df_quote.apply(lambda row: row['End position'] - row['Start position'], axis=1)

In [19]:
df_quote['word_count'] = df_quote.apply(lambda row: len(row['Text Content'].split()), axis=1)

In [20]:
df_quote['max_thread_len'] = df_quote.groupby(['Document'])['word_count'].transform(max)
df_quote['max_comment_len'] = df_quote.groupby(['Document', 'time_cur', 'author_cur'])['word_count'].transform(max)
df_quote['percent_len_thread'] = df_quote.apply(lambda row: row['word_count']/row['max_thread_len'], axis=1)
df_quote['percent_len_comment'] = df_quote.apply(lambda row: row['word_count']/row['max_comment_len'], axis=1)

In [21]:
df_quote['rank_thread'] = df_quote.groupby('Document')['Start position'].rank('dense', ascending=True)
df_quote['rank_comment'] = df_quote.groupby(['Document', 'time_cur', 'author_cur'])['Start position'].rank('dense', ascending=True)
df_quote['max_rank_thread'] = df_quote.groupby(['Document'])['rank_thread'].transform(max)
df_quote['max_rank_comment'] = df_quote.groupby(['Document', 'time_cur', 'author_cur'])['rank_comment'].transform(max)
df_quote['percent_pos_thread'] = df_quote.apply(lambda row: row['rank_thread']/row['max_rank_thread'], axis=1)
df_quote['percent_pos_comment'] = df_quote.apply(lambda row: row['rank_comment']/row['max_rank_comment'], axis=1)

In [22]:
df_quote['is_first_comment'] = df_quote.apply(GetIsFirstComment, axis=1)

df_quote['is_last_comment'] = df_quote.apply(lambda row: row['rank_thread'] == row['max_rank_thread'], axis=1)

In [23]:
def GetDateTimeObjects(row):
    date_time_pre = row['time_pre']
    date_time_cur = row['time_cur']
    date_time_next = row['time_next']
    rex = re.compile("^\d\d\d\d-(0?[1-9]|1[0-2])-(0?[1-9]|[12][0-9]|3[01])T(00|0[0-9]|1[0-9]|2[0-3]):([0-9]|[0-5][0-9]):([0-9]|[0-5][0-9])Z$")
    if (not pd.isna(date_time_cur)) and rex.match(date_time_cur) != None:
        cur_datetime_object = datetime.strptime(date_time_cur, datetime_format_github)
        if (not pd.isna(date_time_pre)) and rex.match(date_time_pre) != None:
            pre_datetime_object = datetime.strptime(date_time_pre, datetime_format_github)
        else:
            pre_datetime_object = cur_datetime_object
        if (not pd.isna(date_time_next)) and rex.match(date_time_next) != None:
            next_datetime_object = datetime.strptime(date_time_next, datetime_format_github)
        else:
            next_datetime_object = cur_datetime_object
    else:
#         print(row['Text Content'])
        print("cur " + date_time_cur)
        cur_datetime_object = datetime.min
        pre_datetime_object = cur_datetime_object
        next_datetime_object = cur_datetime_object
    return pre_datetime_object, cur_datetime_object, next_datetime_object
    

In [24]:
df_quote[['pre_datetime_object', 'cur_datetime_object', 'next_datetime_object']] = df_quote.apply(GetDateTimeObjects, axis=1, result_type="expand")

df_quote['start_datetime_object'] = df_quote.groupby(['Document'])['cur_datetime_object'].transform(min)
df_quote['end_datetime_object'] = df_quote.groupby(['Document'])['cur_datetime_object'].transform(max)

df_quote['duration_thread'] = df_quote.apply(lambda row: (row['end_datetime_object'] - row['start_datetime_object']).total_seconds(), axis=1)
df_quote['duration_start_to_cur'] = df_quote.apply(lambda row: (row['cur_datetime_object'] - row['start_datetime_object']).total_seconds(), axis=1)
df_quote['duration_cur_to_end'] = df_quote.apply(lambda row: (row['end_datetime_object'] - row['cur_datetime_object']).total_seconds(), axis=1)
df_quote['duration_pre_to_cur'] = df_quote.apply(lambda row: (row['cur_datetime_object'] - row['pre_datetime_object']).total_seconds(), axis=1)
df_quote['duration_cur_to_next'] = df_quote.apply(lambda row: (row['next_datetime_object'] - row['cur_datetime_object']).total_seconds(), axis=1)

df_quote['percent_start_time_thread'] = df_quote.apply(lambda row: row['duration_start_to_cur']/row['duration_thread'], axis=1)
df_quote['percent_end_time_thread'] = df_quote.apply(lambda row: row['duration_cur_to_end']/row['duration_thread'], axis=1)
df_quote['percent_previous_time_thread'] = df_quote.apply(lambda row: row['duration_pre_to_cur']/row['duration_thread'], axis=1)
df_quote['percent_next_time_thread'] = df_quote.apply(lambda row: row['duration_cur_to_next']/row['duration_thread'], axis=1)

In [25]:
df_comment = df_quote.groupby(['author_cur', 'time_cur', 'Document'])

In [26]:
def ContainCode(row):
    author = row['author_cur']
    time = row['time_cur']
    document = row['Document']
    same_comment = df_comment.get_group((author, time, document))
    for comment in same_comment['Text Content'].tolist():
        if (('CODE_SEGMENT' in comment) | ('CODE_BLOCK' in comment)):
            return True
    return False

In [27]:
df_quote['contain_code'] = df_quote.apply(ContainCode, axis=1)

In [28]:
pd.set_option('display.max_colwidth', -1)

display(df_quote)

,Text Content,Document,Codes,Start position,End position,time_pre,author_pre,time_cur,author_cur,time_next,...,duration_thread,duration_start_to_cur,duration_cur_to_end,duration_pre_to_cur,duration_cur_to_next,percent_start_time_thread,percent_end_time_thread,percent_previous_time_thread,percent_next_time_thread,contain_code
0,"Is it possible to add feature in order to force vscode to open file only by double clicking on it, single click should only select a file in explorer ?",issue9388_allCommentsText.doc,"9388_claim 1, description of expected behaviour",393,545,NaN,NaN,2016-07-16T14:48:59Z,ivalexa,2016-07-16T16:20:03Z,...,48733127.0,0.0,48733127.0,0.0,5464.0,0.000000,1.000000,0.000000,0.000112,False
1,This can be very useful when there is a lot of files and you looking the exact one in explorer.,issue9388_allCommentsText.doc,"9388_warrant 1, feature motivation and use cases",546,641,NaN,NaN,2016-07-16T14:48:59Z,ivalexa,2016-07-16T16:20:03Z,...,48733127.0,0.0,48733127.0,0.0,5464.0,0.000000,1.000000,0.000000,0.000112,False
2,"I'm using VS Code VERSION_NUM Insider, but I think VERSION_NUM behaves the same way: single-click just to select and preview the file; double-click already open it.",issue9388_allCommentsText.doc,"9388_irrelevant, discuss issue source",760,913,2016-07-16T14:48:59Z,ivalexa,2016-07-16T16:20:03Z,brunnopleffken,2016-07-16T16:44:34Z,...,48733127.0,5464.0,48727663.0,5464.0,1471.0,0.000112,0.999888,0.000112,0.000030,False
3,otherwise there is no reason for double click,issue9388_allCommentsText.doc,"9388_warrant 1, clarification of expected behaviour",1079,1124,2016-07-16T16:20:03Z,brunnopleffken,2016-07-16T16:44:34Z,ivalexa,2016-07-16T22:02:25Z,...,48733127.0,6935.0,48726192.0,1471.0,19071.0,0.000142,0.999858,0.000030,0.000391,False
4,"SCREEN_NAME The 'preview' function does not keep the file open when you preview or open another file in the same editor stack (unless you edit the previewed file, which makes the file opened).",issue9388_allCommentsText.doc,"9388_data 1, 9388_rebuttal 1, discuss issue source",1233,1423,2016-07-16T16:44:34Z,ivalexa,2016-07-16T22:02:25Z,bgse,2016-07-21T04:55:50Z,...,48733127.0,26006.0,48707121.0,19071.0,370405.0,0.000534,0.999466,0.000391,0.007601,False
5,"Not sure what you are going for here, why would you actually (left-)click the file if you do not intend to do anything with it?",issue9388_allCommentsText.doc,"9388_rebuttal 1, 9388_warrant 1, question about motivation and use cases",1426,1553,2016-07-16T16:44:34Z,ivalexa,2016-07-16T22:02:25Z,bgse,2016-07-21T04:55:50Z,...,48733127.0,26006.0,48707121.0,19071.0,370405.0,0.000534,0.999466,0.000391,0.007601,False
6,I also am trying to figure out how to do this,issue9388_allCommentsText.doc,"9388_warrant 1, small talk",1665,1710,2016-07-16T22:02:25Z,bgse,2016-07-21T04:55:50Z,payonel,2016-08-23T04:17:22Z,...,48733127.0,396411.0,48336716.0,370405.0,2848892.0,0.008134,0.991866,0.007601,0.058459,False
7,"because I want the explorer panel to have focus, I might then use the arrow keys to scroll up and down in the list of files for me.",issue9388_allCommentsText.doc,"9388_warrant 1, feature motivation and use cases",1718,1849,2016-07-16T22:02:25Z,bgse,2016-07-21T04:55:50Z,payonel,2016-08-23T04:17:22Z,...,48733127.0,396411.0,48336716.0,370405.0,2848892.0,0.008134,0.991866,0.007601,0.058459,False
8,I was hoping I could type the name of a file and it's scroll to the next file it found starting with that letter.,issue9388_allCommentsText.doc,"9388_warrant 1, feature motivation and use cases",1947,2060,2016-07-16T22:02:25Z,bgse,2016-07-21T04:55:50Z,payonel,2016-08-23T04:17:22Z,...,48733127.0,396411.0,48336716.0,370405.0,2848892.0,0.008134,0.991866,0.007601,0.058459,False
9,even with KB focus in the explorer window it doesn't do this.,issue9388_allCommentsText.doc,"9388_data 1, feature motivation and use cases",2076,2137,2016-07-16T22:02:25Z,bgse,2016-07-21T04:55:50Z,payonel,2016-08-23T04:17:22Z,...,48733127.0,396411.0,48336716.0,370405.0,284889

In [29]:
header = ['Text Content', 'Document', 'Codes',
          'author_association', 'is_poster', 
          'quote_len', 'percent_len_thread', 
          'percent_len_comment', 'percent_pos_thread', 
          'percent_pos_comment', 'is_first_comment',
          'is_last_comment', 'percent_start_time_thread',
          'percent_end_time_thread', 'percent_previous_time_thread',
          'percent_next_time_thread', 'contain_code']

df_quote.to_csv('Conversational_data.csv', columns=header, index=False)